# Optimal Power flow

#### Parameter Definitions

In our data file, we have lots of values, and multiple sheets. Take a moment to make sure you understand what every parameter means, conceptually. Below you can find the relevant parameter definitions.

##### Node (i.e. Bus) Data
**Parameter Name** | **Description**
---|---
Vmax        | Maximal voltage of node
Vmin        | Minimal voltage of node
VAnglemax   | Maximal voltage angle of node
VAnglemin   | Minimal voltage angle of node
Pload       | Real power drawn by loads at node
Qload       | Reactive power drawn by loads at node
PGmax       | Maximal real power of generators at node
PGmin       | Minimal real power of generators at node
QGmax       | Maximal reactive power of generators at node
QGmin       | Minimal reactive power of generators at node
a           | Generator cost function first coefficient# a*p^2
b           | Generator cost function second coefficient# b*p
c           | Generator cost function third coefficient# c

where the cost function of a dispatchable generator at node $i$, with a generation power of $p_i$, is defined as

$$ C_i(p_i) = a_i p_i^2 + b_i p_i + c_i $$

subject to the power constraints $ P^{G,min}_i \le p_i \le P^{Gmax}_i $.

**Note**: The slack node, i.e. the reference node, serves as an angular reference for all other nodes in the system, and is thus set to 0 deg. At this slack node, the voltage magnitude is also assumed to be 1.0 p.u.

##### Admittance Matrices

To model the admittance between two buses, i and k, we have set up the following matrices.
One matrix is for the real part of the admittance, while the other is for the imaginary part.
Since these matrices cover all relationships between buses, these are square matrices.
An example for the structure of the Real Admittance Matrix is shown below:

$$\begin{bmatrix}
G_{1,1} & G_{1,2} & \dots & \dots & G_{1,n} \\
G_{2,1} & \ddots & & & \vdots \\
\vdots & & G_{i,k} & & \vdots \\
\vdots & & & \ddots & G_{n-1,n} \\
G_{n,1} & \dots & \dots & G_{n,n-1} & G_{n,n} 
\end{bmatrix}$$

**Knowledge Checkup:**
To make sure you understand the data structure, take a look at the data file(s) and see if you can answer the following conceptual questions:

1) How many buses are there in total?
2) Which buses have generators connected to them?
3) Which buses have loads connected to them?
4) Which one of these nodes is the slack node?

After you have answered these questions for yourself, you're all ready to code!

#### Dependency Setup

In [48]:
import os
import numpy as np
import pandas as pd
import pyomo.environ as pyo

from opf_utils import solve_and_print, create_results_json

# TODO: Your email address (required for NEOS)
os.environ['NEOS_EMAIL'] = 'piotr.reszel@tum.de'
FILENAME = 'data/OPFData.xlsx'

#### Model Setup

In [49]:
# filename = filename of the data source
# m = model
def add_node_data(filename, m):
    # TODO: Load the pandas dataframes for the node data.
    # Hint: Pay attention to the file types and headers. 
    data = pd.read_excel(filename,sheet_name = 'NodeData')

    # TODO: Set the number of nodes
    m.nodes = pyo.Set(initialize=range(0,5))
    
    # TODO: Complete this function
    m.V_max = data['Vmax']
    m.V_min = data['Vmin']
    m.V_angle_max =data['VAnglemax']
    m.V_angle_min = data['VAnglemin']
    m.P_load = data['Pload']
    m.Q_load = data['Qload']
    m.P_G_max = data['PGmax']
    m.P_G_min = data['PGmin']
    m.Q_G_max = data['QGmin']
    m.Q_G_min = data['QGmin']
    m.a = data['a']
    m.b = data['b']
    m.c = data['c']

    return m

def add_real_admittance_data(filename, m):
    # TODO: Load the pandas dataframes for the node data.
    # Hint: Pay attention to the file types and headers.
    data = pd.read_excel(filename,sheet_name = 'RealAdmittanceMatrix',header=None)
    assert data.shape[0] == data.shape[1], "The admittance data is not a square matrix"

    # TODO: Complete this function
    m.G = data
    return m

def add_imag_admittance_data(filename, m):
    # TODO: Load the pandas dataframes for the node data.
    # Hint: Pay attention to the file types and headers.
    data = pd.read_excel(filename,sheet_name = 'ImaginaryAdmittanceMatrix',header=None)
    assert data.shape[0] == data.shape[1], "The admittance data is not a square matrix"

    # TODO: Complete this function
    m.B = data
    return m

In [50]:
pd.read_excel(FILENAME,sheet_name = 'RealAdmittanceMatrix',header=None)

,0,1,2,3,4
0,1.07,0.00,-1.07,0.00,0
1,0.00,5.66,0.00,-5.66,0
2,-1.07,0.00,1.41,-0.09,0
3,0.00,-5.66,-0.49,5.95,0
4,0.00,0.00,0.00,0.00,0


#### AC Optimal Power Flow Setup

In [63]:
def objective_rule(m):
    cost_ext_grid = m.b[0]*m.x_p[0]
    cost_gen_1 = m.a[2]*m.x_p[2]**2 +m.b[2]*m.x_p[2]+m.c[2]
    cost_gen_2 = m.a[3]*m.x_p[3]**2 +m.b[3]*m.x_p[3]+m.c[3]
    total_cost = cost_ext_grid + cost_gen_1 + cost_gen_2 
    return total_cost

# # m = model
# # i = node index
def power_bounds_rule_1(m, i):   
    return m.x_p[i] <= m.P_G_max[i] # Eq. 1.4

def power_bounds_rule_2(m, i):
    return m.x_p[i] >= m.P_G_min[i]
    
def voltage_angle_bounds_rule_1(m, i):
    return m.x_v_angle[i] <= m.V_angle_max[i] # Eq. 1.7

def voltage_angle_bounds_rule_2(m, i):
    return m.x_v_angle[i] >= m.V_angle_min[i]

def reactive_power_bounds_rule_1(m, i):
    return m.x_q[i] <= m.Q_G_max[i] # Eq. 1.7

def reactive_power_bounds_rule_2(m, i):    
    return m.x_q[i] >= m.Q_G_min[i]
    
def voltage_bounds_rule_1(m, i):
    cons_1 = m.x_v[i] <= m.V_max[i]
    return  m.x_v[i] <= m.V_max[i]

def voltage_bounds_rule_2(m, i):                                
    return  m.x_v_angle[i] >= m.V_min[i]

def power_line_flow_rule_p(m,i):
    p = 0
    for k in range(5):
        p += m.x_v[k]*(m.G.iloc[i,k]*pyo.cos(m.x_v_angle[i]-m.x_v_angle[k])+m.B.iloc[i,k]*pyo.sin(m.x_v_angle[i]-m.x_v_angle[k]))
    p = m.x_v[i]*p
    return m.x_p_line[i] == p

def power_line_flow_rule_q(m,i):
    q = 0
    for k in range(5):
        q += m.x_v[k]*(m.G.iloc[i,k]*pyo.sin(m.x_v_angle[i]-m.x_v_angle[k])-m.B.iloc[i,k]*pyo.cos(m.x_v_angle[i]-m.x_v_angle[k]))
    q = q*m.x_v[i]
    return m.x_q_line[i] == q

def power_balance_rule_ac(m, i):
    return m.x_p[i]-m.P_load[i]-m.x_p_line[i] == 0 # Eq. 1.2

def reactive_power_balance_rule(m, i):
    return m.x_q[i]-m.Q_load[i]-m.x_q_line[i] == 0 # Eq. 1.2

def add_linear_constraints_ac(m):
    # TODO: Complete this function
    m.power_bounds_1 = pyo.Constraint(m.nodes,rule =power_bounds_rule_1) # Using power_bounds_rule
    m.voltage_angle_bounds_1 = pyo.Constraint(m.nodes,rule =voltage_angle_bounds_rule_1) # Using voltage_angle_bounds_rule
    m.reactive_power_bounds_1 = pyo.Constraint(m.nodes,rule =reactive_power_bounds_rule_1) # Using reactive_power_bounds_rule
    m.voltage_bounds_1 = pyo.Constraint(m.nodes,rule =voltage_bounds_rule_1) # Using voltage_bounds_rule
    m.power_bounds_2 = pyo.Constraint(m.nodes,rule =power_bounds_rule_2) # Using power_bounds_rule
    m.voltage_angle_bounds_2 = pyo.Constraint(m.nodes,rule =voltage_angle_bounds_rule_2) # Using voltage_angle_bounds_rule
    m.reactive_power_bounds_2 = pyo.Constraint(m.nodes,rule =reactive_power_bounds_rule_2) # Using reactive_power_bounds_rule
    m.voltage_bounds_2 = pyo.Constraint(m.nodes,rule =voltage_bounds_rule_2) # Using voltage_bounds_rule
    return m

def add_nonlinear_constraints_ac(m):
    m.power_balance = pyo.Constraint(m.nodes,rule =power_balance_rule_ac) # Using power_balance_rule_ac
    m.reactive_power_balance = pyo.Constraint(m.nodes,rule =reactive_power_balance_rule) # Using reactive_power_balance_rule
    m.line_flow_p = pyo.Constraint(m.nodes,rule =power_line_flow_rule_p)
    m.line_flow_q = pyo.Constraint(m.nodes,rule =power_line_flow_rule_q)
    return m

##### DC Optimal Power Flow Setup

In [64]:
# m = model
# i = node index

def power_bounds_rule_1(m, i):   
    return m.x_p[i] <= m.P_G_max[i] # Eq. 1.4

def power_bounds_rule_2(m, i):
    return m.x_p[i] >= m.P_G_min[i]

def voltage_angle_bounds_rule_1(m, i):
    return m.x_v_angle[i] <= m.V_angle_max[i] # Eq. 1.7

def voltage_angle_bounds_rule_2(m, i):
    return m.x_v_angle[i] >= m.V_angle_min[i]

def power_line_flow_rule_p(m,i):
    
    p=0
    for k in range(m.B.shape[1]):
        p+= m.B.iloc[i,k]*(m.x_v_angle[i]-m.x_v_angle[k])
        
    return m.x_p_line[i] == p

def power_balance_rule_dc(m, i):

    return m.x_p[i]-m.P_load[i]-m.x_p_line[i] == 0
        
def add_linear_constraints_dc(m):

    m.power_bounds_1 = pyo.Constraint(m.nodes,rule = power_bounds_rule_1) # Using power_bounds_rule
    m.voltage_angle_bounds_1 = pyo.Constraint(m.nodes,rule = voltage_angle_bounds_rule_1) # Using voltage_angle_bounds_rule
    m.power_bounds_2 = pyo.Constraint(m.nodes,rule = power_bounds_rule_2) # Using power_bounds_rule
    m.voltage_angle_bounds_2 = pyo.Constraint(m.nodes,rule = voltage_angle_bounds_rule_2) # Using voltage_angle_bounds_rule

    return m

def add_nonlinear_constraints_dc(m):

    m.power_balance = pyo.Constraint(m.nodes,rule = power_balance_rule_dc) # Using power_balance_rule_dc
    
    return m

##### Load Model from Data

In [65]:
# Loads the data from the given file into the model, defines the optimization problem, and returns the model at the end.
def load_model(filename, m, AC=True):
    # Fill the model with the parameters from the files
    m = add_node_data(filename, m)
    m = add_real_admittance_data(filename, m)
    m = add_imag_admittance_data(filename, m)

    # Define optimization variables
    m.x_p_line = pyo.Var(m.nodes,
                         within=pyo.Reals)
    m.x_q_line = pyo.Var(m.nodes,
                         within=pyo.Reals)
    m.x_p = pyo.Var(m.nodes,
                    within=pyo.Reals,
                    doc='Optimization variable (power generation)')
    m.x_q = pyo.Var(m.nodes,
                    within=pyo.Reals,
                    doc='Optimization variable (reactive power generation)')
    m.x_v = pyo.Var(m.nodes,
                    within=pyo.Reals,
                    doc='Optimization variable (voltage)')
    m.x_v_angle = pyo.Var(m.nodes,
                         within=pyo.Reals,
                         doc='Optimization variable (voltage angle)')
    
    # Define objective function
    m.objective_function = pyo.Objective(
        rule=objective_rule,
        sense=pyo.minimize,
        doc='minimize(cost = sum of all generator operating costs)')
    
    # Define the linear constraints
    if AC:
        m = add_linear_constraints_ac(m)
    else:
        m = add_linear_constraints_dc(m)
    
    # Define the nonlinear constraints
    if AC:
        m = add_nonlinear_constraints_ac(m)
    else:
        m = add_nonlinear_constraints_dc(m)

    return m

##### Solve AC Model

In [54]:
AC = True
model = load_model(FILENAME, pyo.ConcreteModel(), AC)
solve_and_print(model, AC)
# TODO: Uncomment if you want to write the result to json file
# team_name = '' # Your team name, ex. 'Group_A'
# create_results_json(model, team_name, AC)

model.name="unknown";
    - termination condition: infeasible
    - message from solver: CONOPT 3.17A\x3a Locally infeasible; objective
      0.46336976000879637; 5 iterations; evals\x3a nf = 4, ng = 3, nc = 5, nJ
      = 5, nH = 0, nHv = 0



Solution


Node		Voltage angle [deg]
1 		 1.0363894627827472
2 		 0.9621102606496302
3 		 1.0224657311238006
4 		 0.9539171588575375
5 		 0.95

Node		Voltage magnitude
1 		 0.0
2 		 0.0
3 		 0.0
4 		 0.0
5 		 0.0

Node		Power
1 		 0.34105645716798955
2 		 0.0
3 		 0.4
4 		 0.4
5 		 0.0

Node		Reactive Power
1 		 0.0
2 		 0.0
3 		 0.0
4 		 0.4000000000000001
5 		 0.129

Cost:  0.46336976000879637


In [55]:
#m.pprint()

##### Solve DC Model

In [66]:
AC = False
model = load_model(FILENAME, pyo.ConcreteModel(), AC)
solve_and_print(model, AC)
# TODO: Uncomment if you want to write the result to json file
# team_name = '' # Your team name, ex. 'Group_A'
# create_results_json(model, team_name, AC)




Solution


Node		Voltage angle [deg]
1 		 0.0
2 		 -3.14159265358979
3 		 -3.14159265358979
4 		 -3.14159265358979
5 		 -3.14159265358979

Node		Power
1 		 -1000.0
2 		 0.0
3 		 0.1
4 		 0.05
5 		 0.0

Cost:  -349.95975


In [57]:
data = pd.read_excel(FILENAME,sheet_name = 'NodeData')

In [58]:
data

,Vmax,Vmin,VAnglemax,VAnglemin,Pload,Qload,PGmax,PGmin,QGmax,QGmin,a,b,c
0,1.00,1.00,0.000000,0.000000,0.000,0.000,1000.0,-1000.00,1000.0,-1000.0,0.0,0.35,0
1,1.05,0.95,3.141593,-3.141593,0.000,0.000,0.0,0.00,0.0,0.0,0.0,0.00,0
2,1.05,0.95,3.141593,-3.141593,0.000,0.000,0.4,0.10,0.3,-0.2,0.4,0.20,0
3,1.05,0.95,3.141593,-3.141593,0.900,0.400,0.4,0.05,0.2,-0.2,0.5,0.30,0
4,1.05,0.95,3.141593,-3.141593,0.239,0.129,0.0,0.00,0.0,0.0,0.0,0.00,0


In [59]:
    m.V_max = data['Vmax']
    m.V_min = data['Vmin']
    m.V_angle_max =data['VAnglemax']
    m.V_angle_min = data['VAnglemin']
    m.P_load = data['Pload']
    m.Q_load = data['Qload']
    m.P_G_max = data['PGmax']
    m.P_G_min = data['PGmin']
    m.Q_G_max = data['QGmin']
    m.Q_G_min = data['QGmin']
    m.a = data['a']
    m.b = data['b']
    m.c = data['c']
    m.nodes = pyo.Set(initialize=range(0,5))

NameError: name 'm' is not defined

In [ ]:
m =pyo.ConcreteModel()

In [ ]:
m.x_p = pyo.Var(m.nodes,
     within=pyo.Reals,
     doc='Optimization variable (power generation)')

In [ ]:
m.x_p.value

In [ ]:
m.x_p[2].value